In [1]:
# 모델 불러오기
import os
import sys
import random
import math
import numpy as np
import skimage.io

from model.mrcnn import utils
import model.mrcnn.model as modellib
from model.mrcnn import visualize
import cv2
from PIL import Image
from model.coco import coco2
import warnings

import json
import io
import requests

ROOT_DIR = os.path.abspath("./") #현재 경로
sys.path.append(ROOT_DIR)

MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# 모델 불러오기    
class InferenceConfig(coco2.CocoConfig): # 'coco.py'의 'CocoConfig' 클래스
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
config = InferenceConfig()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Using TensorFlow backend.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [2]:
def background_masking(model, input,test_folder_path): # 
        out_list = []
        # COCO Class names
        class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                    'bus', 'train', 'truck', 'boat', 'traffic light',
                    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                    'kite', 'baseball bat', 'baseball glove', 'skateboard',
                    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                    'teddy bear', 'hair drier', 'toothbrush']


        # 'test_folder' 내부의 파일 목록을 가져옴

        for filename in input:
            print(filename)
            
            image = skimage.io.imread(os.path.join(test_folder_path, filename))

            # Check if the image has 4 channels (RGBA) and convert it to RGB
            if image.shape[2] == 4:
                image = Image.fromarray(image)
                image = image.convert('RGB')
                image = np.array(image)

            results= model.detect([image],verbose=0)
            
            r = results[0]

            # 임계값 설정
            threshold = 0.9

            indices_to_keep = np.where(r['scores'] > threshold)[0]

            # 선택한 인덱스에 해당하는 masks, scores, class_ids를 가져옵니다.
            r['masks'] = r['masks'][:, :, indices_to_keep]
            r['scores'] = r['scores'][indices_to_keep]
            r['class_ids'] = r['class_ids'][indices_to_keep]

            masks = r['masks'][:, :, r['class_ids'] == 1]  # 사람인 경우
            
            if not masks.size > 0:
                print('NO person!!')
                out_list.append(filename)
            else:
                mask = np.sum(masks, axis=2).astype(np.bool)  # 채널 하나짜리 마스크
                # 확장된 마스크 생성
                kernel = np.ones((25, 25), np.uint8)  # 팽창 연산에 사용될 커널 설정
                expanded_mask = cv2.dilate(mask.astype(np.uint8), kernel, iterations=1)
                expanded_mask = expanded_mask.astype(np.bool)
                mask_3d = np.repeat(np.expand_dims(expanded_mask, axis=2), 3, axis=2).astype(np.uint8)  # 채널 3짜리 마스크

                # 이미지 블러 처리
                blurred_img = cv2.GaussianBlur(image, (101, 101), 125)
                mask_3d_blurred = (cv2.GaussianBlur(mask_3d*255,(101,101),10,10)/255).astype(np.float32)

                # mix it together
                person_mask = mask_3d_blurred * image.astype(np.float32)
                bg_mask = (1 - mask_3d_blurred) * blurred_img.astype(np.float32)
                out = (person_mask + bg_mask).astype(np.uint8)
                out_image = out.astype(np.uint8)
                
                bgr_image = cv2.cvtColor(out_image, cv2.COLOR_RGB2BGR) # 최종본!
                print(filename, 'finish!') # 잘되는지 안되는지 확인
                
                #img_name = 'img'+ str(NUM) + '.jpg'
                output_path = os.path.join(test_folder_path,'back', filename) #위에서 정의한 경로에 filname으로 저장하기 위한 코드
                cv2.imwrite(output_path, bgr_image)# 아웃풋 폴
            
                out_list.append(output_path)
                print('output_path',output_path)

        return out_list

In [5]:
import os

# 현재 디렉토리 위치
current_directory = os.getcwd()

# 이미지 파일을 저장할 배열
image_names = []

# 'test_folder' 경로
test_folder_path = os.path.join(current_directory, 'test_folder')

# 'test_folder' 내부의 파일 목록을 가져옴
if os.path.exists(test_folder_path) and os.path.isdir(test_folder_path):
    image_names = [f for f in os.listdir(test_folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]
image_names
test_folder_path

'c:\\Users\\hOMe_pc\\Code\\AIs\\server\\FlaskTest\\test_folder'

In [6]:
out_list = background_masking(model, image_names, test_folder_path)

a1.jpeg
a1.jpeg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\a1.jpeg
c1.jpg
c1.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\c1.jpg
c2.jpg
c2.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\c2.jpg
ive1.jpg
ive1.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\ive1.jpg
j.jpg
j.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\j.jpg
jan1.jpg
jan1.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jan1.jpg
jo1.jpg
jo1.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jo1.jpg
jo2.png
jo2.png finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jo2.png
jo3.jpg
jo3.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jo3.jpg
jo4.jpg
jo4.jpg finish!
output_path c:\Users\hOMe_pc\Code\AIs\server\F

In [1]:
from __future__ import division
import torch
import os
import cv2
import numpy as np
from face_model.common.utils import BBox, drawLandmark, drawLandmark_multiple
from face_model.models.basenet import MobileNet_GDConv
from face_model.blur.bluring import total_blur
from PIL import Image
from face_model.MTCNN import detect_faces
from face_model.utils.align_trans import get_reference_facial_points, warp_and_crop_face

import json
import io
import requests


if torch.cuda.is_available():
    map_location = lambda storage, loc: storage.cuda()
else:
    map_location = 'cpu'
    
model_f = MobileNet_GDConv(136)
model_f = torch.nn.DataParallel(model_f)
checkpoint_path = 'face_model/checkpoint/mobilenet_224_model_best_gdconv_external.pth.tar'
checkpoint = torch.load(checkpoint_path, map_location=map_location)
print(map_location)
print('Use MobileNet as backbone')
model_f.load_state_dict(checkpoint['state_dict'])

c:\Users\hOMe_pc\Anaconda3\envs\AIS_mrcnn\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\hOMe_pc\Anaconda3\envs\AIS_mrcnn\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


cpu
Use MobileNet as backbone


<All keys matched successfully>

In [2]:
import os

# 현재 디렉토리 위치
current_directory = os.getcwd()

# 이미지 파일을 저장할 배열
image_names = []

# 'test_folder' 경로
test_folder_path = os.path.join(current_directory, 'test_folder')

# 'test_folder' 내부의 파일 목록을 가져옴
if os.path.exists(test_folder_path) and os.path.isdir(test_folder_path):
    image_names = [f for f in os.listdir(test_folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]
image_names
test_folder_path

'c:\\Users\\hOMe_pc\\Code\\AIs\\server\\FlaskTest\\test_folder'

In [3]:
from face_model.MTCNN import detect_faces


def face_masking(model, input ,test_folder_path): #
    mean = np.asarray([0.485, 0.456, 0.406])
    std = np.asarray([0.229, 0.224, 0.225])
    out_size = 224
    crop_size = 112
    scale = crop_size / 112.
    reference = get_reference_facial_points(default_square=True) * scale
    model = model.eval()
    # 이미지 파일 순회 및 처리
    for file_name in input:
        print(os.path.join(test_folder_path, file_name))
        img = cv2.imread(os.path.join(test_folder_path, file_name))
        
        
        height, width, _ = img.shape
        ##detector
        image = Image.open(os.path.join(test_folder_path, file_name))
        # Check if the image has 4 channels (RGBA) and convert it to RGB
        if image.mode == "RGBA":
            image = image.convert("RGB")
        
        faces, landmarks = detect_faces(image)


        if len(landmarks) == 0:
            print('NO face is detected!')
            
        if len(faces) == 0:
            print('NO face is detected!')
            
        else:
            # Initialize an empty list to store landmarks
            all_landmarks = []

            for k, face in enumerate(faces):
                if face[4] < 0.9:  # remove low confidence detection
                    continue
                x1 = face[0]
                y1 = face[1]
                x2 = face[2]
                y2 = face[3]
                w = x2 - x1 + 1
                h = y2 - y1 + 1
                size = int(min([w, h]) * 1.2)
                cx = x1 + w // 2
                cy = y1 + h // 2
                x1 = cx - size // 2
                x2 = x1 + size
                y1 = cy - size // 2
                y2 = y1 + size

                dx = max(0, -x1)
                dy = max(0, -y1)
                x1 = max(0, x1)
                y1 = max(0, y1)

                edx = max(0, x2 - width)
                edy = max(0, y2 - height)
                x2 = min(width, x2)
                y2 = min(height, y2)
                new_bbox = list(map(int, [x1, x2, y1, y2]))
                new_bbox = BBox(new_bbox)
                cropped = img[new_bbox.top:new_bbox.bottom, new_bbox.left:new_bbox.right]
                if (dx > 0 or dy > 0 or edx > 0 or edy > 0):
                    cropped = cv2.copyMakeBorder(cropped, int(dy), int(edy), int(dx), int(edx), cv2.BORDER_CONSTANT, 0)
                cropped_face = cv2.resize(cropped, (out_size, out_size))

                if cropped_face.shape[0] <= 0 or cropped_face.shape[1] <= 0:
                    continue
                test_face = cropped_face.copy()
                test_face = test_face / 255.0
                test_face = (test_face - mean) / std
                test_face = test_face.transpose((2, 0, 1))
                test_face = test_face.reshape((1,) + test_face.shape)
                input = torch.from_numpy(test_face).float()
                input = torch.autograd.Variable(input)
                
                landmark = model(input).cpu().data.numpy()
                
                landmark = landmark.reshape(-1, 2)
                landmark = new_bbox.reprojectLandmark(landmark)
                
                # Append the landmarks to the list
                all_landmarks.append(landmark)
            
            # Draw landmarks on the original image
            for landmarks in all_landmarks:
                
                img = total_blur(img, landmarks)
            
             #img_name = 'img'+ str(NUM) + '.jpg'
            output_path = os.path.join(test_folder_path,'face', file_name) #위에서 정의한 경로에 filname으로 저장하기 위한 코드
            cv2.imwrite(output_path, img)# 아웃풋 저장 덮어씌우기
    
    return 

In [4]:
face_masking(model_f, image_names ,test_folder_path)

c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\a1.jpeg
NO face is detected!
NO face is detected!
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\c1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\c2.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\ive1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\j.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jan1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo2.png
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo3.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo4.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo5.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\jo6.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\lee1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\lee3.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\new1.jpg
c:\Users\hOM

In [13]:
import os

# 현재 디렉토리 위치
current_directory = os.getcwd()

# 이미지 파일을 저장할 배열
image_names = []

# 'test_folder' 경로
test_folder_path = os.path.join(current_directory, 'test_folder','back')

# 'test_folder' 내부의 파일 목록을 가져옴
if os.path.exists(test_folder_path) and os.path.isdir(test_folder_path):
    image_names = [f for f in os.listdir(test_folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]
image_names
test_folder_path

'c:\\Users\\hOMe_pc\\Code\\AIs\\server\\FlaskTest\\test_folder\\back'

In [15]:
from face_model.MTCNN import detect_faces


def face_masking(model, input ,test_folder_path): #
    mean = np.asarray([0.485, 0.456, 0.406])
    std = np.asarray([0.229, 0.224, 0.225])
    out_size = 224
    crop_size = 112
    scale = crop_size / 112.
    reference = get_reference_facial_points(default_square=True) * scale
    model = model.eval()
    # 이미지 파일 순회 및 처리
    for file_name in input:
        print(os.path.join(test_folder_path, file_name))
        img = cv2.imread(os.path.join(test_folder_path, file_name))

        
        height, width, _ = img.shape
        ##detector
        image = Image.open(os.path.join(test_folder_path, file_name))
        # Check if the image has 4 channels (RGBA) and convert it to RGB
        if image.mode == "RGBA":
            image = image.convert("RGB")
        
        faces, landmarks = detect_faces(image)


        if len(landmarks) == 0:
            print('NO face is detected!')
            
        if len(faces) == 0:
            print('NO face is detected!')
            
        else:
            # Initialize an empty list to store landmarks
            all_landmarks = []

            for k, face in enumerate(faces):
                if face[4] < 0.9:  # remove low confidence detection
                    continue
                x1 = face[0]
                y1 = face[1]
                x2 = face[2]
                y2 = face[3]
                w = x2 - x1 + 1
                h = y2 - y1 + 1
                size = int(min([w, h]) * 1.2)
                cx = x1 + w // 2
                cy = y1 + h // 2
                x1 = cx - size // 2
                x2 = x1 + size
                y1 = cy - size // 2
                y2 = y1 + size

                dx = max(0, -x1)
                dy = max(0, -y1)
                x1 = max(0, x1)
                y1 = max(0, y1)

                edx = max(0, x2 - width)
                edy = max(0, y2 - height)
                x2 = min(width, x2)
                y2 = min(height, y2)
                new_bbox = list(map(int, [x1, x2, y1, y2]))
                new_bbox = BBox(new_bbox)
                cropped = img[new_bbox.top:new_bbox.bottom, new_bbox.left:new_bbox.right]
                if (dx > 0 or dy > 0 or edx > 0 or edy > 0):
                    cropped = cv2.copyMakeBorder(cropped, int(dy), int(edy), int(dx), int(edx), cv2.BORDER_CONSTANT, 0)
                cropped_face = cv2.resize(cropped, (out_size, out_size))

                if cropped_face.shape[0] <= 0 or cropped_face.shape[1] <= 0:
                    continue
                test_face = cropped_face.copy()
                test_face = test_face / 255.0
                test_face = (test_face - mean) / std
                test_face = test_face.transpose((2, 0, 1))
                test_face = test_face.reshape((1,) + test_face.shape)
                input = torch.from_numpy(test_face).float()
                input = torch.autograd.Variable(input)
                
                landmark = model(input).cpu().data.numpy()
                
                landmark = landmark.reshape(-1, 2)
                landmark = new_bbox.reprojectLandmark(landmark)
                
                # Append the landmarks to the list
                all_landmarks.append(landmark)
            
            # Draw landmarks on the original image
            for landmarks in all_landmarks:
                
                img = total_blur(img, landmarks)
            
             #img_name = 'img'+ str(NUM) + '.jpg'
            output_path = os.path.join(test_folder_path,'total', file_name) #위에서 정의한 경로에 filname으로 저장하기 위한 코드
            cv2.imwrite(output_path, img)# 아웃풋 저장 덮어씌우기
            print(output_path)
    
    return 

In [16]:
face_masking(model_f, image_names ,test_folder_path)

c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\a1.jpeg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\a1.jpeg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\c1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\c1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\c2.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\c2.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\ive1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\ive1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\j.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\j.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jan1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\jan1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\jo1.jpg
c:\Users\hOMe_pc\Code\AIs\server\FlaskTest\test_folder\back\total\jo1.jpg
c:\Users\h